<a href="https://colab.research.google.com/github/kenttw/kent-ai-learning-notebook/blob/master/Machine_Learning_Exercise/CreditCard/SimpleModel_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# if you have google drive permission then use this 

from google.colab import drive
drive.mount('/gdrive')
!ln -s   /gdrive/MyDrive/colab/Credit_Card/data/ /content/data

In [ ]:
!pip3 install gdown
!mkdir -p ./data/
import gdown


url = 'https://drive.google.com/uc?id=%s'%("1NZ8jMoqcCDpydHxWm6j415HwVS093l4U")
output = './data/X_sample.csv'
gdown.download(url, output, quiet=False)


url = 'https://drive.google.com/uc?id=%s'%("18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI")
output = './data/y_sample.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy")
output = './data/X_test.csv'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=%s'%("1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG")
output = './data/y_test.csv'
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1NZ8jMoqcCDpydHxWm6j415HwVS093l4U
To: /content/data/X_sample.csv
48.9MB [00:00, 230MB/s]
Downloading...
From: https://drive.google.com/uc?id=18w2LpyPbydD9qoVQ33GZI2LOjyRY2LaI
To: /content/data/y_sample.csv
100%|██████████| 288k/288k [00:00<00:00, 49.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-0TD5rSfzG3frbgT4fseGBn-Dn5OpGKy
To: /content/data/X_test.csv
609MB [00:03, 188MB/s]
Downloading...
From: https://drive.google.com/uc?id=1-1p1dHQVV2JnGP4eMfzJsQwErum_wuMG
To: /content/data/y_test.csv
3.59MB [00:00, 207MB/s]


'./data/y_test.csv'

In [ ]:
import pandas as pd

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score

In [ ]:
df_X = pd.read_csv("./data/X_sample.csv")
df_X = df_X.set_index("txkey")

In [ ]:
df_y = pd.read_csv("./data/y_sample.csv")
df_y = df_y.set_index("txkey")

In [ ]:
import lightgbm as lgb


In [ ]:
from sklearn.model_selection import train_test_split
df_X_train, df_X_val = train_test_split(df_X, test_size=0.25, random_state=42)


In [ ]:
df_y_train = df_y.loc[df_X_train.index]
df_y_val = df_y.loc[df_X_val.index]

In [ ]:
# create dataset for lightgbm
# lgb_train = lgb.Dataset(df_X_train, df_y_train)
# lgb_eval = lgb.Dataset(df_X_val, df_y_val, reference=lgb_train)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(max_depth=2, random_state=0)
clf.fit(df_X_train, df_y_train)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=2, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

# Validation with downsampleing data

In [ ]:
print('Starting predicting...')
# predict
y_pred = clf.predict_proba(df_X_val)[:,1]
print('auc:', roc_auc_score(df_y_val, y_pred))

Starting predicting...
auc: 0.9461556034447327


In [ ]:
confusion_matrix(df_y_val, (y_pred > 0.5).astype(int))

array([[3360,  500],
       [ 420, 3362]])

In [ ]:
tn, fp, fn, tp  = confusion_matrix(df_y_val, (y_pred > 0.5).astype(int)).ravel() 

In [ ]:
tn, fp, fn, tp

(3360, 500, 420, 3362)

# Testing : verify with real distribution data

In [ ]:
X_test = pd.read_csv("./data/X_test.csv")

In [ ]:
X_test = X_test.set_index("txkey")

In [ ]:
y_test = pd.read_csv("./data/y_test.csv")

In [ ]:
y_test = y_test.set_index("txkey")

In [ ]:
y_pred = clf.predict_proba(X_test)[:,1]


In [ ]:
print('auc:', roc_auc_score(y_test, y_pred))

auc: 0.9414470406039809


In [ ]:
cm = confusion_matrix(y_test, (y_pred > 0.5).astype(int), labels=[0,1])
cm

array([[323377,  51998],
       [   583,   4489]])

In [ ]:
th = .6
print("precision_score", precision_score(y_test,(y_pred > th).astype(int)))
print("recall_score",    recall_score(y_test,(y_pred > th).astype(int)))
print("f1_score",        f1_score(y_test,(y_pred > th).astype(int)))

precision_score 0.13404561909113274
recall_score 0.7519716088328076
f1_score 0.22753169276659208


0.22753169276659208